# Mock Computing Paper
**Produce of Qian Yi.**<br>Find the instructions in the attached PDF file.<br>Not that deep, but good luck still.

<table>
<tr>
    <td>TOTAL MARKS</td><td>55</td>
</tr>
<tr>
    <td>DURATION</td><td>2 HOURS</td>
</tr>
</table>
<br>

<hr>

## TASK 1.1
To store each game entry in Python, implement the `Entry` class according to the UML diagram. **[8]**

In [1]:
"""
This helper string may be convenient be you.
{"player_id":..., "player_name":..., "score":..., "timestamp":...}
"""

class Entry:
    def __init__(self, player_id: str, player_name: str, score: str | int, timestamp: str):
        self._player_id: str = player_id
        self._player_name: str = player_name
        self._timestamp: str = timestamp

        try:
            self._score = int(score)
        except ValueError:
            self._score = 0
    
    def get_player_id(self): return self._player_id
    def get_player_name(self): return self._player_name
    def get_score(self): return self._score
    def get_timestamp(self): return self._timestamp

    def output(self) -> dict:
        return {
            "player_id": self.get_player_id(),
            "player_name": self.get_player_name(),
            "score": self.get_score(),
            "timestamp": self.get_timestamp(),
        }
    
    def __str__(self) -> str:
        return f"{self.get_player_name()} ({self.get_player_id()}) - {self.get_score()} @ {self.get_timestamp()}"
    
    def __lt__(self, other) -> bool:
        return (self.get_score(), self.get_timestamp()) < \
               (other.get_score(), other.get_timestamp())

In [2]:
def test_1_1():
    entry_1 = Entry("S-2025-002", "Drake", 690, "2025/09/02 12:00:00")
    entry_2 = Entry("S-2025-003", "Lamar", "690", "2025/08/31 23:59:59")
    entry_3 = Entry("S-2025-001", "Diddy", "abc", "2025/09/01 00:00:01")
    
    print(entry_2.output())
    # {'player_id': 'S-2025-003', 'player_name': 'Lamar', 'score': 690, 'timestamp': '2025/08/31 23:59:59'}

    print(entry_3)
    # Diddy (S-2025-001) - 0 @ 2025/09/01 00:00:01
    
    print(entry_3 < entry_1)    # True
    print(entry_2 < entry_1)    # True
    print(entry_1 < entry_1)    # False
    print(entry_2 < entry_3)    # False

test_1_1()

{'player_id': 'S-2025-003', 'player_name': 'Lamar', 'score': 690, 'timestamp': '2025/08/31 23:59:59'}
Diddy (S-2025-001) - 0 @ 2025/09/01 00:00:01
True
True
False
False


<hr>

## TASK 1.2** (updated)

Implement `merge_sort(entries: list[Entry])` to sort a list of Entries by **Merge Sort**, such that it is in **descending order**. [3]

In [3]:
def merge_sort(entries: list) -> list:
    if len(entries) <= 1:
        return entries
    mid = len(entries) // 2
    left = merge_sort(entries[:mid])
    right = merge_sort(entries[mid:])
    return merge(left, right)

def merge(l1: list, l2: list) -> list:
    result = []
    while not (len(l1) == 0 or len(l2) == 0):
        if l1[0] > l2[0]:
            result.append(l1.pop(0))
        else:
            result.append(l2.pop(0))
    return result + l1 + l2

In [4]:
def test_1_2():
    entries = [
        Entry("S-2025-001", "Alpha", 500, "2025/09/02 12:00:00"),
        Entry("S-2025-002", "Beta", 990, "2024/01/01 00:00:01"),
        Entry("S-2025-003", "Gamma", 1000, "2025/02/28 14:03:05"),
        Entry("S-2025-004", "Delta", 990, "2023/11/09 23:59:57"),
        Entry("S-2025-006", "Epsilon", "abc", "2025/06/07 06:07:42"),
        Entry("S-2025-007", "Zeta", 500, "2025/09/02 12:00:01"),
    ]

    # Gamma, Beta, Delta, Zeta, Alpha, Epsilon
    for entry in merge_sort(entries):
        print(entry.get_player_name(), end=", ")

test_1_2()

Gamma, Beta, Delta, Zeta, Alpha, Epsilon, 

<hr>

## TASK 1.3
You are given `messy.csv`. Clean up the data, sort it then produce `clean.txt` per the specifications. **[8]**

In [5]:
raws = []
with open("data_files/messy.csv") as f:
    count_x = -1
    for line in f:
        count_x += 1
        if count_x != 0:  # Skip the header
            raws.append(line.strip())

proper = []
for row in raws:
    data = row.split(",")
    # Ignore empty names & IDs
    if "" in (data[0], data[1]):    
        continue
    data[1] = data[1].title()
    # Try to convert score to integer, else ignore
    try:    
        data[2] = int(data[2])
    except ValueError:
        continue
    # Replace slashes with dashes, and add seconds
    data[3] = data[3].replace("/", "-")
    if len(data[3].split(":")) < 3:
        data[3] += ":00"

    proper.append(Entry(data[0], data[1], data[2], data[3]))

# Sort, then find counts
proper = merge_sort(proper)

count_y = count_x - len(proper)
scores = [entry.get_score() for entry in proper]
count_z = round(sum(scores)/len(scores))

# Output
with open("clean.txt", "w") as f:
    f.write(f"Entries processed: {count_x}\n")
    f.write(f"Entries removed: {count_y}\n")
    f.write(f"Average rounded Score: {count_z}\n")
    for entry in proper:
        f.write(str(entry) + "\n")

<hr>

## TASK 2.1
In Python, generate `tombzone.db` and the tables: `Player`, `Game`, `Entry`. The CREATE statements are to include checks, non-nulls, defaults, types and key references where appropriate. **[6]**

In [6]:
import sqlite3
DB_FILE = "tombzone.db"

query_1 = """CREATE TABLE IF NOT EXISTS "Player" (
	"player_id"	TEXT NOT NULL UNIQUE,
	"name"	TEXT NOT NULL,
	"gender"	INTEGER NOT NULL CHECK(gender IN ('M', 'F')),
	"hp"	TEXT NOT NULL,
	PRIMARY KEY("player_id")
)"""
query_2 = """CREATE TABLE IF NOT EXISTS "Game" (
	"game_id"	INTEGER NOT NULL UNIQUE,
	"game_name"	TEXT NOT NULL UNIQUE,
	"max_score"	INTEGER NOT NULL DEFAULT 1000,
	PRIMARY KEY("game_id" AUTOINCREMENT)
)"""
query_3 = """CREATE TABLE IF NOT EXISTS "Entry" (
	"entry_id"	INTEGER NOT NULL UNIQUE,
	"player_id"	TEXT NOT NULL,
	"game_id"	INTEGER NOT NULL,
	"score"	INTEGER NOT NULL CHECK(score >= 0),
	"timestamp"	TEXT NOT NULL,
	PRIMARY KEY("entry_id" AUTOINCREMENT),
	FOREIGN KEY("game_id") REFERENCES "Game"("game_id"),
	FOREIGN KEY("player_id") REFERENCES "Player"("player_id")
)"""

conn = sqlite3.connect(DB_FILE)
for query in (query_1, query_2, query_3):
    conn.execute(query)
conn.commit()
conn.close()

<hr>

## TASK 2.2

Populate the corresponding tables in `tombzone.db` with files `players.csv`, `games.csv` and `entries.csv`, taking note of duplicate entries. **[7]**

(Hint: To ignore duplicates, use INSERT OR IGNORE INTO)

In [7]:
import csv
TABLE_FILES: dict = {
    "Player": "players.csv",
    "Game": "games.csv",
    "Entry": "entries.csv"
}

conn = sqlite3.connect(DB_FILE)
for table in TABLE_FILES:
    with open(f"data_files/{TABLE_FILES[table]}") as f:
        reading = csv.reader(f)
        header = next(reading)
        placeholders = ",".join(['?' for _ in header])
        for row in reading:
            conn.execute(
                f"""INSERT OR IGNORE INTO {table} {tuple(header)} VALUES ({placeholders})""",
                tuple(row)
            )
            
conn.commit()
conn.close()

<hr>

## TASK 2.3

Implement `investigate(game_name: str, gender: str)` per the specifications. **[5]**

(Hint: To divide X by Y in SQL, you can do `(X*1.0)/Y` to cast X as a floating point to be divisible by Y. To round numbers in SQL, use `ROUND(number, decimal_places)`)

A snippet output for the first 2 entries of female players in “Clash Royole” is as follows:
| name | hp | timestamp | score_percentage |
| --- | --- | --- | --- |
| Zoey | 88385346 | 2025-09-03 16:25:00 | 0.3 |
| Ella O'Brien | 58488556 | 2025-09-01 16:05:00 | 0.15 |

In [8]:
def investigate(game_name: str, gender: str):
    # To evaluate the score percentage,
    # Cast entry score to integer, divide, then round to 2 d.p.
    # When comparing by score_percentage,
    # Just compare by score, since max_score is the same
    query = """
    SELECT p.name, p.hp, e.timestamp,
    ROUND((e.score*1.00)/g.max_score, 2) AS "score_percentage"
    FROM Game g
    INNER JOIN Entry e ON e.game_id = g.game_id
    INNER JOIN Player p ON p.player_id = e.player_id
    WHERE game_name = ?
    AND gender = ?
    ORDER BY e.score DESC, e.timestamp ASC
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.execute(
        query,
        (game_name, gender)
    )
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return data

In [9]:
def test_2_3():
    print(" | name | hp | timestamp | score_percentage |")
    for row in investigate("Closh Royole", "F"):
        print("", *row, "", sep=" | ")

test_2_3()

 | name | hp | timestamp | score_percentage |
 | Zoey | 88385346 | 2025-09-03 16:25:00 | 0.3 | 
 | Ella O'Brien | 58488556 | 2025-09-01 16:05:00 | 0.15 | 
 | Zoey | 88385346 | 2025-09-01 14:10:00 | 0.13 | 
 | Ginna | 98343378 | 2025-09-03 12:00:00 | 0.1 | 
 | Bobby Lim | 31829585 | 2025-09-03 13:50:00 | 0.09 | 
 | Qwertinius | 88835466 | 2025-09-03 14:00:00 | 0.09 | 


<hr>

Complete **Task 2.4** outside of this notebook with the relevant files parked under the `task_2_4/` folder.

**END OF PAPER.**